# Nedensel Görevlerin Gerçekleştirilmesi

Nedensel ilişkileri modelledikten sonra bir sonraki adıma geçmeye hazırız: nedensel görevleri yerine getirmek. DoWhy aşağıdaki görevleri destekler.

* Etki tahmini: X'i değiştirirsek Y'nin ne kadar değişmesine neden olur?
* Yükleme (attribution): Bir olay neden meydana geldi? Bir sonuç nasıl açıklanır? Değişkenlerimden hangisi anormalliğe neden oldu?
* Karşıolgusal tahmin: Ya X, gözlemlenen değerinden farklı bir değere değiştirilmiş olsaydı? Diğer değişkenlerin değerleri ne olurdu?
* Tahmin: Bazı girdi özniteliklerin yeni değerlerine sahip bir girdi verildiğinde Y'nin değeri ne olacaktır?

# 1. Nedensel Etki Tahmini

![PCM](../img/dowhy-schematic.png)

Bir $A$ değişkeninin $Y$ üzerindeki nedensel etkisi, $A$'daki bir değişiklik nedeniyle $Y$'da beklenen değişiklik olarak tanımlanır. Do-calculus gösterimini kullanarak, ortalama nedensel etki şu şekilde yazılabilir: $E[Y \mid do(A)]$. Bazen yalnızca bir alt popülasyon üzerindeki nedensel etkiyle ilgileniriz veya nedensel etkiyi alt popülasyonlar arasında karşılaştırmak isteriz. Bu durumda, $X, E[Y \mid d o(A), X]$ ortak değişkenleri kümesi verildiğinde koşullu ortalama nedensel etkiyi (CACE) tahmin edebiliriz.

Nedensel etkiyi tahmin etmek dört adım gerektirir:
1. Varsayımları kullanarak nedensel bir çıkarım problemini modelleme.
2. Bu varsayımlar altında nedensel etki için bir ifade belirleme ("nedensel tahmin").
3. Eşleştirme veya araçsal değişkenler gibi istatistiksel yöntemleri kullanarak ifadeyi tahmin etme.
4. Son olarak çeşitli sağlamlık kontrolleri (robustness checks) kullanarak tahminin geçerliliğini doğrulama.

Bu iş akışı DoWhy'daki dört anahtar metod tarafından içerilmektedir:
- model (CausalModel veya graph)
- tanımlama (identify_effect)
- tahmin (estimate_effect)
- çürütme (refute_estimate)

DoWhy, bu metodları kullanarak çeşitli yöntemleri destekleyen bir nedensel etki tahmin API'si uygular. *Model*, ön bilgiyi (prior knowledge) formel bir nedensel çizge olarak kodlar; *tanımlama*, nedensel etkiyi tanımlamak için çizge tabanlı yöntemler kullanır; *tahmin*, belirlenen tahminin tahmin edilmesi için istatistiksel yöntemler kullanır ve son olarak *çürütme*, varsayımların sağlamlığını test ederek elde edilen tahmini çürütmeye çalışır. Bu nedenle nedensel çizgeyi oluşturduktan sonra nedensel etkiyi tahmin etmenin bir sonraki adımı, etkinin mevcut verilerden tahmin edilip edilemeyeceğini belirlemektir. Başka bir deyişle, bir tahmin algoritmasını dikkate almadan önce bir tanımlama stratejisi belirlemek önemlidir. DoWhy aşağıdaki tanımlama algoritmalarını destekler:
- Arka kapı (backdoor)
- Ön kapı (frontdoor)
- Araç değişken (instrumental variable)
- ID algoritması (ID algorithm)

Nedensel etki belirlendikten sonra tanımlama stratejisiyle uyumlu bir tahmin yöntemi seçebiliriz. Ortalama nedensel etkiyi tahmin etmek için DoWhy aşağıdaki yöntemleri destekler:

* Methods based on matching confounders' values:
  - Distance-based matching (DistanceMatchingEstimator)
*  Methods based on estimating the treatment assignment
  - Propensity-based Stratification (PropensityScoreStratificationEstimator)
  - Propensity Score Matching (PropensityScoreMatchingEstimator)
  - Inverse Propensity Weighting (PropensityScoreWeightingEstimator)
*  Methods based on estimating the outcome model
  - Linear Regression (LinearRegressionEstimator)
  - Generalized Linear Models, including logistic regression (GeneralizedLinearModelEstimator)
* Methods based on the instrumental variable equation
  - Binary Instrument/Wald Estimator (InstrumentalVariableEstimator)
  - Regression discontinuity(RegressionDiscontinuityEstimator)
* Methods for front-door criterion and mediation analysis
  - Two-stage linear regression (TwoStageRegressionEstimator)

Koşullu ortalama nedensel etkiyi tahmin etmek için DoWhy, EconML yöntemlerinin çağrılmasını destekler.  $\mathrm{Y}$ sonucu için veri oluşturma süreci doğrusal bir fonksiyon olarak tahmin edilebiliyorsa, CACE tahmini için doğrusal regresyon yöntemini de kullanabilirsiniz.

## 1.1. Nedensel Etkinin Tanımlanması

Nedensel bir çizge ve gözlemlenen değişkenler kümesi göz önüne alındığında, nedensel etkinin belirlenmesi, etkinin mevcut değişkenlerin verileri kullanılarak tahmin edilip edilemeyeceğinin belirlenmesi sürecidir. Formel olarak, tanımlama, hedef nedensel etki ifadesini örneğin $E[Y \mid do(A)]$ alır ve bunu gözlemlenen veri dağılımı kullanılarak, yani do-operatörü olmadan tahmin edilebilecek bir forma dönüştürür.

### 1.1.1. Arka Kapı (Backdoor) Kriteri

Nedensel etkiyi arka kapı kriteri ile tanımlamak için şunu yazabiliriz:

```python
# model is an instance of CausalModel
identified_estimand = model.identify_effect()
print(identified_estimand)
```

Yukarıdaki kod, varsayılan bir arka kapı ayarlamasını kullanır. Dört temel tür arka kapı ayarı mevcuttur. Bunların her biri geçerli bir arka kapı kümesi döndürmek üzere tasarlanmıştır, ancak döndürülecek değişken kümesini nasıl seçtiklerine göre farklılık gösterirler.

1. *maximal-adjustment*: arka kapı kriterini karşılayan maksimum seti döndürür. Bu genellikle geçerli bir arka kapı seti bulmanın en hızlı yoludur, ancak set birçok gereksiz değişken içerebilir.
2. *minimal-adjustemnt*: arka kapı kriterini karşılayan minimum sayıda değişken içeren seti döndürür. Bunun yürütülmesi daha uzun sürebilir ve bazen maksimum yineleme sayısı dahilinde herhangi bir arka kapı seti döndürülmeyebilir.
3. *exhaustive-search*: tüm geçerli arka kapı kümelerini döndürür. Büyük grafiklerde bunun çalıştırılması biraz zaman alabilir.
4. *default*: Bu, minimum ve maksimum ayarlamanın iyi bir karışımıdır. Genellikle hızlı olan maksimum ayarlamayla başlar. Daha sonra minimum ayarlamayı çalıştırır ve en az sayıda değişkene sahip olan seti döndürür.

Belirli bir tür arka kapı ayarlamasını kullanmak için *method* argümanını kullanabiliriz.

```python
identified_estimand = model.identify_effect(method="exhaustive-search")
print(identified_estimand)
```

*identify_effect* yönteminin diğer kimlik türlerini de arayan kullanışlı bir yöntem olduğunu unutmayın. Aşağıda işlevsel API'yi kullanan eşdeğer bir çağrı yer almaktadır.

```python
from dowhy.causal_identifier import identify_effect_auto, BackdoorAdjustment
identified_estimand_auto = identify_effect_auto(
   graph,
   treatment_name,
   outcome_name,
   backdoor_adjustment=BackdoorAdjustment.BACKDOOR_MAXIMAL
)
print(identified_estimand_auto)
```

### 1.1.2. Ön Kapı (Frontdoor) Kriteri

Ön kapı kriterini kullanarak etkiyi tanımlamak için aynı *identify_effect* yöntemini kullanırız. Arka kapı, ön kapı ve araçsal değişkenler arasındaki olası tüm tanımlama stratejilerini kontrol eder.

```python
# model is an instance of CausalModel
identified_estimand = model.identify_effect()
print(identified_estimand)
```

### 1.1.3. Doğal Deneyler ve Araç Değişkenler (Instrumental Variables)

Araç değişkenleri kullanarak etkiyi tanımlamak için aynı *identify_effect* yöntemini kullanırız. Arka kapı, ön kapı ve araçsal değişkenler arasındaki olası tüm tanımlama stratejilerini kontrol eder.

```python
# model is an instance of CausalModel
identified_estimand = model.identify_effect()
print(identified_estimand)
```

### 1.1.4. Yeni Tanımlama Stratejileri Keşfetmek için ID Algoritması

ID algoritması (Shpitser & Pearl 2006), nedensel etkinin tanımlanmasına yönelik gelişmiş bir algoritmadır. ID algoritmasını kullanmak için yöntem adını belirtebilirsiniz,

```python
# model is an instance of CausalModel
identified_estimand = model.identify_effect(method="id-algorithm")
print(identified_estimand)
```

Alternatif olarak fonksiyonu doğrudan çağırabiliriz.

```python
from dowhy.causal_identifier import identify_effect_id
identified_estimand_id = identify_effect_id(
       graph, treatment_name, outcome_name,
)
# Note that the return type for id_identify_effect is IDExpression and not IdentifiedEstimand
print(identified_estimand)
```

## 1.2. Ortalama Nedensel Etkinin Arka Kapı ile Tahmin Edilmesi

Arka kapıyla etki tahmini, koşullu olasılık dağılımının tahmin edilmesine eşdeğerdir. Bir A eylemi, $\mathrm{Y}$ sonucu ve $\mathrm{W}$ arka kapı değişkenleri kümesi verildiğinde, nedensel etki şu şekilde tanımlanır: $\sum_w E[Y \mid A, W=w] P(W=w)$

Koşullu beklentinin çıktısını alabilecek herhangi bir tahminciyi kullanabiliriz. DoWhy, ortalama nedensel etki için aşağıdaki üç tür tahminciyi destekler:

* Regresyon tabanlı metodlar
* Uzaklık tabanlı eşleştirme (Distance-based matching)
* Eğilim tabanlı eşleştirme (Propensity-based matching)
* Do-sampler

### 1.2.1. Regresyon Tabanlı Metodlar

Doğrusal regresyon nedensel etkiyi tahmin etmek için en yaygın yöntemlerden biridir. Y sonucu için veri oluşturma sürecinin doğrusal bir fonksiyon olarak tahmin edilebilmesi faydalıdır.

Arka kapı tanımlı bir tahmin göz önüne alındığında, doğrusal regresyon kullanarak nedensel etkiyi tahmin etmek için şunu yazabiliriz:

```python
estimate = model.estimate_effect(identified_estimand,
       method_name="backdoor.linear_regression",
       test_significance=True
)
print(estimate)
```
Yukarıdaki yöntem, modelin uydurulmasını ve nedensel etkinin tahmin edilmesini birleştirir. Daha fazla kontrol elde etmek için işlevsel API'yi kullanabiliriz.

```python
# Fit the regression estimator
estimator = LinearRegressionEstimator(
   identified_estimand=identified_estimand,
   test_significance=True,
).fit(
   data=data["df"],
   effect_modifier_names=graph.get_effect_modifiers(treatment_name, outcome_name),
)
# Estimate the effect given treatment and control value
estimate = estimator.estimate_effect(
   data=data["df"],
   control_value=0,
   treatment_value=1,
   target_units="ate",
)
print(estimate)
```

Doğrusal regresyonun yanı sıra DoWhy genelleştirilmiş doğrusal modelleri de destekler. Bu, lojistik regresyon modeline uymak için kullanılabilir.

### 1.2.2. Uzaklık Tabanlı Eşleştirme

```python	
causal_estimate_dmatch = model.estimate_effect(identified_estimand,
                                             method_name="backdoor.distance_matching",
                                             target_units="att",
                                             method_params={'distance_metric':"minkowski", 'p':2})
print(causal_estimate_dmatch)
```

### 1.2.3. Eğilim Tabanlı Eşleştirme

Propensity-based methods are backdoor estimation methods that involve estimating the action as a function of the backdoor variables, $P(A \mid W)$. This fitted function is then used to derive matching, stratification or weighting methods.

Eğilim tabanlı eşleştirme:

```python
causal_estimate_match = model.estimate_effect(identified_estimand,
                                             method_name="backdoor.propensity_score_matching",
                                             target_units="atc")
print(causal_estimate_match)
print("Causal Estimate is " + str(causal_estimate_match.value))
```

Eğilim tabanlı sınıflandırma:

```python
causal_estimate_strat = model.estimate_effect(identified_estimand,
                                             method_name="backdoor.propensity_score_stratification",
                                             target_units="att")
print(causal_estimate_strat)
print("Causal Estimate is " + str(causal_estimate_strat.value))
```

Eğilim tabanlı ağırlıklandırma:

```python
causal_estimate_ipw = model.estimate_effect(identified_estimand,
                                           method_name="backdoor.propensity_score_weighting",
                                           target_units = "ate",
                                           method_params={"weighting_scheme":"ips_weight"})
print(causal_estimate_ipw)
print("Causal Estimate is " + str(causal_estimate_ipw.value))
```

### 1.2.4. Do-Sampler

Potansiyel çıktılara (potential outcomes) dayalı tahmincilerin çoğu belirli karşıtlığı (contrast) tahmin etmeye odaklanırken, Pearlian çıkarımı (Pearlian inference), diğer ilgilenilen istatistikleri türetmek için kullanılabilen $\mathrm{Y}$ sonuçlarının ortak dağılımı gibi daha temel niceliklere odaklanır.

Genellikle bir olasılık dağılımını parametrik olmayan şekilde temsil etmek zordur. Yapabilseniz bile, onu oluşturmak için kullandığınız verilerle ilgili sonlu örnek problemlerini gözden kaçırmak istemezsiniz. Bu sorunları göz önünde bulundurarak müdahaleses dağılımları (interventional distributions) "do-sampler" adı verilen bir nesneyle örnekleyerek temsil etmeye karar verdik. Bu örneklerle müdahalesel verilerimizin sonlu örnek istatistiklerini hesaplamayı umabiliriz. Bu tür örneklerin çoğunu bootstrap yaparsak, bu istatistikler için iyi örnekleme dağılımları elde etmeyi bile umut edebiliriz.

Kullanıcı bunun hala aktif bir araştırma alanı olduğunu unutmamalıdır, bu nedenle örnekleyicilerin bootstraplı hata aralıklarına fazla güvenmemeye dikkat etmelisiniz. Örnekleyicilerin sonuç dağılımından örnekleme yaptığını ve dolayısıyla örnekten örneğe önemli ölçüde farklılık göstereceğini unutmayın. Sonuçları hesaplamak amacıyla bunları kullanmak için, ilgilendiğiniz istatistiğin sonsal varyansı hakkında fikir edinmek amacıyla bu türden birkaç örnek oluşturmanız önerilir.

***Pearlian Müdahaleler***

Pearlian nedensel modele müdahale fikrini takiben, örnekleyicilerimiz bir dizi adımı uygular:
- Disrupt causes
- Make effective
- Propagate and sample

İlk aşamada, müdahale ettiğimiz tüm değişkenlerin kenarlarını kesmeyi hayal ediyoruz. İkinci aşamada bu değişkenlerin değerini müdahale miktarlarına ayarlıyoruz. Üçüncü aşamada, bir örnekleme prosedürüyle müdahale sonuçlarını hesaplamak için bu değeri modelimiz aracılığıyla ileriye doğru yayıyoruz.

Pratikte bu adımları uygulayabileceğimiz birçok yol var. Modeli PyMC3'te doğrusal bir Bayes ağı olarak oluşturduğumuzda bunlar en açık şekilde ortaya çıkar; bu, MCMC'nin örnekleyicisinin temelini oluşturan şeydir. Bu durumda, verilere bir bayes ağı uydururuz, ardından müdahalesel ağı temsil eden yeni bir ağ oluştururuz. Yapısal denklemler, ilk ağa uygun parametrelerle ayarlanır ve kendi örneğimizi elde etmek için bu yeni ağdan örnekleme yaparız.

Örnekleyicinin ağırlıklandırılmasında, eğilim skoru (propensity score) tahmini yoluyla nedensel duruma yönelik seçimi hesaba katarak soyut olarak "nedenleri bozmayı" düşünüyoruz. Bu puanlar, arka kapı yollarını kapatmak için kullanılan bilgileri içerir ve dolayısıyla nedensel duruma geçişle aynı istatistiksel etkiye sahiptir. Veri setimizin alt kümesini nedensel durumun doğru değerine sahip seçerek tedaviyi etkili hale getiriyoruz. Son olarak, örneğimizi elde etmek için ters eğilim ağırlıklandırmasını kullanarak ağırlıklı rastgele bir örnek oluşturduk.

Bu üç adımı uygulamanın başka yolları da var ancak formül aynı. Kendi örnekleyicinizi oluşturmak istiyorsanız bunları geçersiz kılmanız gereken soyut sınıf yöntemleri olarak soyutladık!

***Durumsallık (Statefulness)***

Yüksek seviye pandas API'si yoluyla erişildiğinde do-sampler varsayılan olarak durum bilgisizdir. Bu, onunla çalışmayı sezgisel hale getirir ve pandas.DataFrame.causal.do'ya tekrarlanan çağrılarla farklı örnekler oluşturabilirsiniz. Durum bilgisi yapılabilir, bu bazen faydalıdır.

Daha önce bahsettiğimiz 3 aşamalı süreç, üç aşamanın her birinden dahili bir pandas.DataFrame geçirilerek ancak geçici olarak kabul edilerek gerçekleştirilir. Sonuç döndürülmeden önce dahili veri çerçevesi varsayılan olarak sıfırlanır.

Örnek oluşturma arasında do-sampler durumu korumak çok daha verimli olabilir. Bu, özellikle MCMC do-sampler, kernel-density-sampler ve weighting-sampler'da olduğu gibi, 1. adım pahalı bir modelin takılmasını gerektirdiğinde geçerlidir. Modeli her örnek için yeniden uydurmak yerine, modeli bir kez uydurmak ve ardından do-samplerdan birçok örnek oluşturmak istersiniz. Pandas.DataFrame.causal.do yöntemini çağırdığınızda kwarg stateful=True ayarını yaparak bunu yapabilirsiniz. Veri çerçevesinin durumunu sıfırlamak için (modelin yanı sıra dahili veri çerçevesinin de silinmesi), pandas.DataFrame.causal.reset yöntemini çağırabilirsiniz.

Alt düzey API aracılığıyla örnekleyici varsayılan olarak durum bilgilidir. Varsayım, düşük seviyeli API kullanan bir "uzman kullanıcının" örnekleme süreci üzerinde daha fazla kontrol isteyeceği yönündedir. Bu durumda durum, örnekleme sırasında iletilen veri çerçevesinin bir kopyası olan dahili veri çerçevesi self._df tarafından taşınır. Orijinal veri çerçevesi self._data'da tutulur ve kullanıcı durumu sıfırladığında kullanılır.

## 1.3. Ortalama Nedensel Etkinin Doğal Deneyler ile Tahmin Edilmesi

Doğal deneyler, araçsal bir değişkenin eylemi etkilediği veya regresyon süreksizliğinin olduğu durumları içerir. Araçsal değişkenler yöntemi, doğal deneyler için ortak bir tanımlama stratejisidir. Aşağıda bu tanımlama stratejisinden türetilen iki tahmin ediciyi gösteriyoruz.

İkili eylem için araçsal değişken tahmincisi:

```python
causal_estimate_iv = model.estimate_effect(identified_estimand,
       method_name="iv.instrumental_variable", method_params = {'iv_instrument_name': 'Z0'})
print(causal_estimate_iv)
print("Causal Estimate is " + str(causal_estimate_iv.value))
```

Regresyon süreksizliği için tahminci:

```python
causal_estimate_regdist = model.estimate_effect(identified_estimand,
       method_name="iv.regression_discontinuity",
       method_params={'rd_variable_name':'Z1',
                      'rd_threshold_value':0.5,
                      'rd_bandwidth': 0.15})
print(causal_estimate_regdist)
print("Causal Estimate is " + str(causal_estimate_regdist.value))
```

## 1.4. Koşullu Ortalama Nedensel Etkinin Tahmin Edilmesi

Koşullu ortalama nedensel etki (CACE) tahmini için Do Why, EconML paketini kullanır. EconML paketindeki tüm yöntemler DoWhy'ın tahmin API'sinden çağrılabilir, böylece tahmin yöntemleri için ortak bir arayüz sağlanır. Ayrıca CACE tahmini için regresyon yöntemleri de kullanılabilir.

CACE tahmini için efekt değiştiricileri doğrudan CausalModel'i çağırırken veya grafiğin bir parçası olarak belirtmemiz gerekir.

Doğrusal regresyon:

```python
linear_estimate = model.estimate_effect(identified_estimand,
                                       method_name="backdoor.linear_regression",
                                      control_value=0,
                                      treatment_value=1)
print(linear_estimate)
```

EconML'den DML metodu:

```python
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LassoCV
from sklearn.ensemble import GradientBoostingRegressor
dml_estimate = model.estimate_effect(identified_estimand, method_name="backdoor.econml.dml.DML",
                                    control_value = 0,
                                    treatment_value = 1,
                                target_units = lambda df: df["X0"]>1,  # condition used for CATE
                                confidence_intervals=False,
                               method_params={"init_params":{'model_y':GradientBoostingRegressor(),
                                                             'model_t': GradientBoostingRegressor(),
                                                             "model_final":LassoCV(fit_intercept=False),
                                                             'featurizer':PolynomialFeatures(degree=1, include_bias=False)},
                                              "fit_params":{}})

```

## 1.5. Ortalama Nedensel Etkinin GCM ile Tahmin Edilmesi

En yaygın nedensel sorulardan biri, belirli bir hedef miktarın iki farklı müdahale/tedavi altında ne kadar farklı olduğudur. Bu aynı zamanda ortalama tedavi etkisi (ATE) veya daha genel olarak ortalama nedensel etki (ACE) olarak da bilinir. En basit biçim, iki tedavinin karşılaştırılmasıdır; yani, verilen A tedavisi ile B tedavisi arasındaki ortalama hedef miktarın arasındaki fark nedir? Örneğin, belirli bir ilaçla tedavi edilen hastalar ( $T:=1$ ), hiç tedavi edilmeyen $(T:=0)$ hastalardan daha hızlı iyileşir mi?. ACE API, bir hedef düğümdeki bu tür farklılıkları tahmin etmeye izin verir, yani $\mathbb{E}[Y \mid \operatorname{do}(T:=A)]-\mathbb{E}[Y \mid \operatorname{do}(T:=B)]$

***Nasıl Kullanılır***

Bunu örnekleyelim:

In [1]:
import warnings
warnings.filterwarnings('ignore')

import networkx as nx
import numpy as np 
import pandas as pd
import dowhy.gcm as gcm

X0 = np.random.normal(0, 0.2, 1000)
T = (X0 > 0).astype(float)
X1 = np.random.normal(0, 0.2, 1000) + 1.5 * T
Y = X1 + np.random.normal(0, 0.1, 1000)
data = pd.DataFrame(dict(T=T, X0=X0, X1=X1, Y=Y))

Burada $T$'nin ikili olduğunu ve 1, aksi halde 0 ise $Y$'a 1,5 eklediğini görüyoruz. Her zamanki gibi neden-sonuç ilişkilerini modelleyelim ve bunu verilere uyduralım:

In [5]:
from dowhy import CausalModel
import dowhy.gcm as gcm

causal_model = gcm.ProbabilisticCausalModel(nx.DiGraph([('X0', 'T'), ('T', 'X1'), ('X1', 'Y')]))
gcm.auto.assign_causal_mechanisms(causal_model, data)
gcm.fit(causal_model, data)

Fitting causal mechanism of node Y: 100%|██████████| 4/4 [00:00<00:00,  9.52it/s] 


Artık şu soruyu yanıtlamaya hazırız: "$T:=1$ ve $T:=0$ ayarının nedensel etkisi nedir?"

In [6]:
gcm.average_causal_effect(causal_model,
                         'Y',
                         interventions_alternative={'T': lambda x: 1},
                         interventions_reference={'T': lambda x: 0},
                         num_samples_to_draw=1000)

Ortalama etki $\sim 1,5$'dır ve bu bizim veri oluşturma sürecimizle örtüşmektedir. Yöntem, müdahalelerin olduğu bir sözlük veri yapısı gerektirdiğinden, birden fazla düğüme de müdahale edebilir ve/veya daha karmaşık müdahaleler belirleyebiliriz.

Pratikte nedensel grafiği doğru şekilde belirlemek zor görünse de, genellikle doğru nedensel sıraya sahip bir grafiği belirtmenin yeterli olduğunu unutmayın. Bu, herhangi bir nedensellik ilişkisi olmadığı sürece, yukarı yöndeki düğümlerden aşağı yöndeki bir düğüme çok fazla kenar eklenmesinin, nedensel etkileri tahmin ederken yine de makul sonuçlar sağlayacağı anlamına gelir. Yukarıdaki örnekte $X 0 \rightarrow Y$ ve $T \rightarrow Y$ kenarını eklersek aynı sonucu elde ederiz:

In [7]:
causal_model.graph.add_edge('X0', 'Y')
causal_model.graph.add_edge('T', 'Y')
gcm.auto.assign_causal_mechanisms(causal_model, data, override_models=True)
gcm.fit(causal_model, data)
gcm.average_causal_effect(causal_model,
                         'Y',
                         interventions_alternative={'T': lambda x: 1},
                         interventions_reference={'T': lambda x: 0},
                         num_samples_to_draw=1000)

Fitting causal mechanism of node Y: 100%|██████████| 4/4 [00:00<00:00, 47.90it/s]


Modellenmemiş kök düğümler arasındaki potansiyel etkileşimleri daha fazla hesaba katmak için yenilerini oluşturmak yerine gözlemsel verileri de iletebiliriz:

In [8]:
gcm.average_causal_effect(causal_model,
                         'Y',
                         interventions_alternative={'T': lambda x: 1},
                         interventions_reference={'T': lambda x: 0},
                         observed_data=data)

***Yöntemin Anlaşılması***

Ortalama nedensel etkiyi tahmin etmek basittir, çünkü bunun yalnızca bir hedef düğümün iki beklentisini ilgili müdahale dağılımlarından alınan örneklere dayalı olarak karşılaştırması gerekir. Yani ACE tahminini aşağıdaki adımlara indirgeyebiliriz:
1. $\mathrm{A}$ tedavisi altındaki $Y$'nin müdahalesel dağılımından örnekler çizilir.
2. Tedavi B kapsamındaki $Y$'nin müdahalesel dağılımından örnekler çizilir.
3. Bunların ortalamalar hesaplanır.
4. Araçların farkları alınır. Bu, $\mathbb{E}[Y \mid \operatorname{do}(T:=A)]-\mathbb{E}[Y \mid \mathrm{do}(T:=B)]$'dir, burada Müdahale türlerini veya müdahale etmek istediğimiz değişkenleri kısıtlamamıza gerek yok.

# 2.Nedensel Etkinin Ölçülmesi

DoWhy, ortalama toplam nedensel etkiyi tahmin etmenin yanı sıra aracılık analizi (mediation analysis), iki düğüm arasındaki doğrudan bağlantı kuvvetini tahmin etmek ve nedensel bir grafikteki düğümlerin içsel nedensel etkisini tahmin etmek için de kullanılabilir.

Bu yöntemler arasındaki temel farklar şunlardır:

**Aracılık Analizi (Mediation Analysis)**: Aracılık analizi öncelikle bir tedavinin sonuç üzerindeki toplam etkisini bir veya daha fazla aracı değişken aracılığıyla doğrudan ve dolaylı etkilere ayırmayı amaçlar. Odak noktası, etkinin ne kadarının doğrudan tedaviden sonuca aktarıldığını ve ne kadarının diğer değişkenler aracılığıyla aracılık ettiğini anlamaktır.

**Doğrudan Bağlantı Gücü (Direct Arrow Strength)**: Bu yöntem, bir çizgedeki bir kenar kaldırıldığında dağılımdaki değişikliği ölçerek bir değişkenin diğeri üzerindeki nedensel etkisini ölçer. Değişikliği tahmin etmek için varyanstaki fark veya bir kenarı kaldırırken göreceli entropi gibi belirli bir ölçü kullanır. Bu, belirli bir nedensel bağlantının kaldırılmasının hedef değişkeni nasıl etkilediğinin anlaşılmasını sağlar ve bu sayede, doğrusal olmayan ilişkiler için iyi tanımlanmış belirli bir nedensel bağlantının gücünü temsil eden tek bir değer sağlar.

**İçsel Nedensel Katkı (Intrinsic Causal Contribution)**: Bu yöntem, atalarından miras alınan etkilerden bağımsız olarak, bir çizgedeki bir düğümün içsel katkısını tahmin etmeye odaklanır. Her bir düğümün yukarı yöndeki gürültü terimlerinin bir fonksiyonu olarak temsil edilmesini içerir ve her bir düğümün gürültü terimlerinin etkisini ölçmek için yapıyı koruyan müdahaleler kullanır. Bu yöntem, her bir düğümün eklediği doğal bilgiyi atalarından elde edilen bilgilerden ayırır. Eklenen bilgiler (koşullu) varyans veya entropi vb. ölçümlerle ölçülebilir.

## 2.1. Aracılık Analizi: Doğal doğrudan ve dolaylı etkilerin tahmin edilmesi

Aracılık analizi, nedensel bir etkinin belirli bir yol aracılığıyla ne ölçüde uygulandığını ölçmek için kullanılabilir. Do Why, *doğal doğrudan etkinin* ve *doğal dolaylı etkinin* tahminini destekler:

* Doğal doğrudan etki: v0->y yolundan kaynaklanan etki 
* Doğal dolaylı etki: v0->FDO->y yolundan kaynaklanan etki (FD0 aracılığıyla).

DoWhy'ın etki tahmin çerçevesini kullanarak estimand_type bağımsız değişkenini buna göre ayarlayarak bir aracılık analizi gerçekleştirebiliriz:

Tanımlama:

```python
# Natural direct effect (nde)
identified_estimand_nde = model.identify_effect(estimand_type="nonparametric-nde",
                                                proceed_when_unidentifiable=True)
print(identified_estimand_nde)
```

```python	
# Natural indirect effect (nie)
identified_estimand_nie = model.identify_effect(estimand_type="nonparametric-nie",
                                                proceed_when_unidentifiable=True)
print(identified_estimand_nie)
```

Tahmin:

```python
import dowhy.causal_estimators.linear_regression_estimator
causal_estimate_nie = model.estimate_effect(identified_estimand_nie,
                                            method_name="mediation.two_stage_regression",
                                            confidence_intervals=False,
                                            test_significance=False,
                                            method_params = {
                                                'first_stage_model': dowhy.causal_estimators.linear_regression_estimator.LinearRegressionEstimator,
                                                'second_stage_model': dowhy.causal_estimators.linear_regression_estimator.LinearRegressionEstimator
                                            })
print(causal_estimate_nie)
```

```python
causal_estimate_nde = model.estimate_effect(identified_estimand_nde,
                                            method_name="mediation.two_stage_regression",
                                            confidence_intervals=False,
                                            test_significance=False,
                                            method_params = {
                                              'first_stage_model': dowhy.causal_estimators.linear_regression_estimator.LinearRegressionEstimator,
                                              'second_stage_model': dowhy.causal_estimators.linear_regression_estimator.LinearRegressionEstimator
                                            })
print(causal_estimate_nde)
```

## 2.2. Doğrudan Etki: Doğrudan Bağlantı Gücünün Tahmin Edilmesi

Bir bağlantının gücünü ölçerek şu soruyu yanıtlıyoruz:Bir nedenden onun doğrudan sonucuna doğru olan nedensel etki ne kadar güçlüdür?

Literatürde nedensel etkileri ölçmek için farklı tanımlar olsa da DoWhy, bir ana düğümün çocuk üzerindeki doğrudan etkisini ölçmek için, diğer düğümler üzerindeki yollardan kaynaklanan etkilerin göz ardı edildiği bir uygulama sunmaktadır. Bu yöntem şu makaleye dayanmaktadır:

Dominik Janzing, David Balduzzi, Moritz Grosse-Wentrup, Bernhard Schölkopf. Quantifying causal influences The Annals of Statistics, Vol. 41, No. 5, 2324-2358, 2013.

***Nasıl Kullanılır***

Öncelikle veri türetelim:

In [9]:
import numpy as np
import pandas as pd
import networkx as nx
from dowhy import gcm
np.random.seed(10)  # to reproduce these results

In [10]:
Z = np.random.normal(loc=0, scale=1, size=1000)
X = 2*Z + np.random.normal(loc=0, scale=1, size=1000)
Y = 3*X + 4*Z + np.random.normal(loc=0, scale=1, size=1000)
data = pd.DataFrame(dict(X=X, Y=Y, Z=Z))

Daha sonra neden-sonuç ilişkilerini Do Why'ın GCM çerçevesini kullanarak olasılıksal bir nedensel model olarak modelleyip verilere uyduracağız:

In [11]:
causal_model = gcm.ProbabilisticCausalModel(nx.DiGraph([('Z', 'Y'), ('Z', 'X'), ('X', 'Y')]))
gcm.auto.assign_causal_mechanisms(causal_model, data)
gcm.fit(causal_model, data)

Fitting causal mechanism of node X: 100%|██████████| 3/3 [00:00<00:00, 750.01it/s]


Son olarak, ilgilenilen bir düğüme (örneğin, $Y$) gelen bağlantıların gücünü tahmin edebiliriz.

In [12]:
strength = gcm.arrow_strength(causal_model, 'Y')
strength

{('X', 'Y'): 40.853078507275704, ('Z', 'Y'): 15.175900631774171}

Sonuçların yorumlanması: Varsayılan olarak, bağlantı kuvvetleri için skaler değerlerin ölçüm birimi, sürekli gerçek değerli bir hedef için varyans ve kategorik bir hedef için (yani KL sapması) bit sayısıdır. Yukarıda, $X$'dan $Y$'a ( $\sim 40.85$) doğrudan etkinin, $Z$'dan $Y$'ye (15.17) doğrudan etkiden ($\sim 2.7$ kat) daha güçlü olduğunu gözlemliyoruz. Kabaca söylemek gerekirse, okun $X$'dan $Y$'a "kaldırılması" $Y$'nin varyansını 40.85 birim artırırken, $Z \rightarrow Y$'ın kaldırılması $Y$'nin varyansını $\sim 15.18$ birim artırır.

Yöntemi anlama bölümünde, bir kenarı "kaldırmanın" ne anlama geldiğini açıklıyoruz. Özellikle bir bağlantının gücünü ölçmeye yönelik yöntemimizin arkasındaki bilimi kısaca açıklıyoruz.

***Yöntemin Anlaşılması***

Yöntemi anlamak için aşağıdaki grafiği kullanacağız:

![PCM](../img/arrow_strength_example.png)

Burada, $Z$ düğümünden $Y$ düğümüne giden bağlantının gücünü ölçmek istiyoruz, bunu yaparken $X$ üzerinden dolaylı etkileri göz ardı ediyoruz. Bunu başarmak için öncelikle $P$ değişkenlerinin ortak dağılımını, ebeveynleri verilen her değişkenin koşullu dağılımlarının bir çarpımı yoluyla nedensel grafiklerinden elde edebileceğimizi hatırlayın.

Daha sonra kenarı $Z$'dan $Y$'a keserek ve bir i.i.d kullanarak yeni bir $P_{Z \rightarrow Y}$ ortak dağılımı yaratırız. $Y$'a girdi olarak $Z$ kopyası (şekilde $Z^{\prime}$ olarak gösterilmiştir). $Z^{\prime}$ dağılımı, gözlemlenen $Z$ değerlerinin rastgele karıştırılmasıyla pratik olarak simüle edilebilir. $Z$ ile $Y$ arasındaki okun kuvveti, $C_{Z \rightarrow Y}$ olarak temsil edilir, daha sonra kesme sonrası dağılım arasındaki mesafe olarak hesaplanır:math: $P_{\_}\{Z $ ila $Y\}$ ve orijinal ortak dağıtım $P$ :

$$
C_{Z \rightarrow Y}:=D\left(P \| P_{Z \rightarrow Y}\right)
$$

Bağlantı kuvvetini hesaplamak için kullanılan uzaklık ölçüsü $D$, varyansların farkı veya KL sapması gibi herhangi bir uygun ölçü olabilir. Varsayılan olarak kitaplık, veri türüne bağlı olarak belirli bir ölçü kullanır ancak bu, kullanım durumuna göre tamamen özelleştirilebilir.

Kenarı keserken, orijinal ve kesme sonrası bağlantı dağılımı arasında yalnızca hedef değişkenin nedensel mekanizmasının değiştiğini unutmayın. Sonuç olarak hedefin marjinal dağılımındaki herhangi bir değişiklik, hedefin nedensel mekanizmasındaki değişiklikten kaynaklanmaktadır. Bu aynı zamanda bir kenarı kaldırdığımızda hedefin marjinal dağılımının özelliğindeki değişiklik (örneğin varyans) açısından okun gücünü ölçmemize de olanak tanır.

Burada uzaklık ölçütü özelleştirilebilir:

In [13]:
def mean_diff(Y_old, Y_new): 
    return np.mean(Y_new) - np.mean(Y_old)

In [14]:
gcm.arrow_strength(causal_model, 'Y', difference_estimation_func=mean_diff)

{('X', 'Y'): 0.13090621837798588, ('Z', 'Y'): -0.07996403913133228}

Buradaki küçük sonuçlar beklenebilir; örneğimizde, gelen bağlantıyı kaldırsak da kaldırmasak da $Y$'ın ortalama değeri 0 olarak kalır. Bu nedenle, $Y$'a gelen bağlantıların ortalama üzerindeki etkisi göz ardı edilebilir düzeyde olmalıdır.

Özetle bağlantı kuvveti farklı birimlerle (örn. ortalama, varyans, bit) ölçülebilir. Bu nedenle, kullanıcılara bu yöntemi pratikte uygulamak için verilere ve yorumlara dayalı olarak anlamlı bir birim seçmelerini tavsiye ediyoruz.

## 2.3. İçsel Nedensel Etkinin Ölçülmesi

İçsel nedensel etkiyi ölçerek şu soruyu yanıtlıyoruz: Yukarı yöndeki bir düğümün, yukarı yöndeki düğümün ebeveynlerinden miras alınmayan bir hedef düğüm üzerindeki nedensel etkisi ne kadar güçlüdür?

Doğal olarak, soyundan gelenlerin hedef düğüm üzerinde sıfır içsel nedensel etkisi olacaktır. Bu yöntem şu makaleye dayanmaktadır:

Dominik Janzing, Patrick Blöbaum, Lenon Minorics, Philipp Faller, Atalanti Mastakouri. Quantifying intrinsic causal contributions via structure preserving interventions arXiv:2007.00714, 2021

Burada ölçülen etkinin türünü anlamak için makaleden bir örnek ele alalım. Tren A, Tren B ve Tren C olmak üzere üç trenden oluşan bir program düşünün; burada C Treninin kalkış saati, Tren B'nin varış saatine bağlıdır ve Tren B'nin kalkış saati, Tren A'nın varış saatine bağlıdır. A, genellikle Tren B ve Tren C'den çok daha uzun gecikmeler yaşar. Cevaplamak istediğimiz soru şudur: Her trenin, Tren C'nin gecikmesi üzerindeki etkisi ne kadar güçlüdür?

Literatürde etkinin çeşitli tanımları olsa da, biz bir düğümün ebeveynlerinden miras alınmamış etkisini, yani bir düğümün gürültüsünün etkisini ölçen içsel nedensel etkiyle ilgileniyoruz. Bunun nedeni C Treni B Trenini beklemek zorunda iken B Treninin gecikmeyi çoğunlukla A Treninden devralmasıdır. Bu nedenle C Treninin gecikmesine en fazla katkı sağlayan düğüm olarak Tren A belirlenmelidir.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import networkx as nx
from dowhy import gcm

In [3]:
X = abs(np.random.normal(loc=0, scale=5, size=1000))
Y = X + abs(np.random.normal(loc=0, scale=1, size=1000))
Z = Y + abs(np.random.normal(loc=0, scale=1, size=1000))
data = pd.DataFrame(dict(X=X, Y=Y, Z=Z))

$X$'deki 'gürültünün' standart sapmasının daha büyük olduğuna dikkat edin.

In [4]:
causal_model = gcm.StructuralCausalModel(nx.DiGraph([('X', 'Y'), ('Y', 'Z')])) # X -> Y -> Z
gcm.auto.assign_causal_mechanisms(causal_model, data)
gcm.fit(causal_model, data)

Fitting causal mechanism of node Z: 100%|██████████| 3/3 [00:00<00:00, 37.73it/s]


Son olarak, ilgi duyduğumuz bir düğüme (örneğin Z) etki eden içsel nedensel etkilerini sorabiliriz.

In [5]:
contributions = gcm.intrinsic_causal_influence(causal_model, 'Z')
contributions

Evaluate set function: 8it [00:00, 7998.67it/s]


{'X': 8.268749492676609, 'Y': 0.3872940792957947, 'Z': 0.41994799082789347}

Burada doğrusal bir ilişki kullanmamıza rağmen, yöntemin keyfi doğrusal olmayan ilişkileri de ele alabileceğini unutmayın.

Sonuçları yorumlama: $Z$'nin öncüllewrinin, kendisi de dahil olmak üzere, varyansına (varsayılan ölçü) içsel nedensel etkisini tahmin ettik. Bu katkılar $Z$'nin varyansında toplanır. Burada gördüğümüz gibi, $Z$'nin varyansının $\sim 92 \%$'sinin $X$'ten geldiğini gözlemliyoruz.

### 2.3.1. İçsel Nedensel Etki Tahmininin Anlaşılması

"İçsel" nedensel etki kavramının ardındaki sezgiyi daha fazla açıklamak için farklı bir örneğe bakalım:

Bir yetimhaneye yardım etmek için para toplamak amacıyla bir yardım etkinliği düzenlenir. Etkinliğin sonunda, her katılımcıya bir bağış kutusu dağıtılır. Bağış gönüllü olduğu için, bazıları çeşitli nedenlerle bağış yapmayabilir. Örneğin, nakitleri olmayabilir. Bu senaryoda, bağış kutusunu diğer katılımcıya veren bir katılımcı sonuçta toplu bağışa hiçbir katkıda bulunmaz. Her kişinin katkısı, bağışladığı miktardır.


Bir kaynak düğümün hedef düğüme içsel nedensel etkisini ölçmek için fonksiyonel bir nedensel modele ihtiyacımız vardır. Örneğin, her düğümün nedensel modelinin bir toplamsal gürültü modelini (additive noise model-ANM) izlediğini varsayabiliriz, yani $X_j:=f_j\left(\mathrm{PA}_j\right)+N_j$ burada $\mathrm{PA}_j$ nedensel grafikteki $X_j$ düğümünün ebeveynleridir ve $N_j$ bağımsız gözlemlenmemiş gürültü terimidir. $X_n$'nin atalarının $X_{n c}$'nin marjinal dağılımının bazı özelliklerine (ör. varyans veya entropi) "içsel" katkısını hesaplamak için önce nedensel grafiğimizi kurmalı ve her düğümün nedensel modelini veri kümesinden öğrenmeliyiz.

Yukarıdaki kod örneğinde olduğu gibi, aşağıdaki ANM'ler tarafından oluşturulan nedensel bir $X \rightarrow Y \rightarrow Z$ grafiğini düşünün.

$$
\begin{aligned}
& X:=N_X \\
& Y:=2 X+N_Y \\
& Z:=3 Y+N_Z
\end{aligned}
$$

burada $N_w \sim \mathcal{N}(0,1)$, tüm $w \in\{X, Y, Z\}$ için, standart normal gürültü değişkenleridir.

Hedef değişken $Z$'nin varyansına her değişkenin katkısıyla ilgilendiğimizi varsayalım, yani $\operatorname{Var}[Z]$. Gürültü değişkenleri olmasaydı, her şey kök düğüm $X$'e katkıda bulunabilirdi çünkü diğer tüm değişkenler o zaman onun deterministik fonksiyonu olurdu. Her değişkenin hedef nicelik $\operatorname{Var}[Z]$'ye içsel katkısı o zaman gerçekten karşılık gelen gürültü teriminin katkısıdır.

"İçsel" katkıyı hesaplamak için, gürültü değişkenlerinin alt kümeleri $N_T$ verildiğinde, yani $P_{Z \mid N_T}$ olduğunda $Z$'nin koşullu dağılımlarına da ihtiyacımız var, burada $T \subseteq\{X, Y, Z\}$. Bunları bir ANM kullanarak tahmin ediyoruz. Bu amaçla, gürültü değişkenlerinin bir alt kümesinden hedef değişkene tahmin modelini belirlememiz gerekiyor. Aşağıda, gürültü değişkenlerinden $Z$'ye doğrusal bir tahmin modeli kullanarak $X, Y$ ve $Z$'nin $\operatorname{Var}[Z]$'ye içsel nedensel etkisini niceliyoruz.

In [7]:
from dowhy.gcm.uncertainty import estimate_variance

predicition_model_from_noises_to_target = gcm.ml.create_linear_regressor()
node_to_contribution = gcm.intrinsic_causal_influence(
    causal_model,
    'Z',
    prediction_model=predicition_model_from_noises_to_target,
    attribution_func=lambda x, _: estimate_variance(x)
)

Evaluate set function: 8it [00:00, 8006.31it/s]


In [8]:
node_to_contribution

{'X': 8.894030853172314, 'Y': 0.32355985380556174, 'Z': 0.3683072069111856}

Burada, attribution_func parametresindeki varyansı, ilgilendiğimiz özellik olarak açıkça tanımladık.

Belirsizlik tahmincisi olarak varyans kullanımı, düğümlerin hedefteki kare sapmalara katkısı hakkında değerli bilgiler verirken, kişi mutlak sapmalar gibi diğer niceliklerle de ilgilenebilir. Bu, attribution_func'ı özel bir işlevle değiştirerek de kolayca hesaplanabilir:

```python
mean_absolute_deviation_estimator = lambda x, y: np.mean(abs(x-y))
node_to_contribution = gcm.intrinsic_causal_influence(causal_model, 'Z',
                                                     prediction_model_from_noises_to_target,
                                                     attribution_func=mean_absolute_deviation_estimator)

```

Tahmin modelinin seçimi belirsiz ise, tahmin modeli parametresi "auto" olarak da ayarlanabilir.

Katkı için ortalamanın kullanılmasına ilişkin açıklama: attribution_func belirli bir kullanım durumu için özelleştirilebilse de, tüm tanımlar mantıklı değildir. Örneğin, ortalamanın kullanılması anlamlı sonuçlar sağlamaz. Bunun nedeni, etkilerin tahmin edilme biçiminin Shapley değerleri kavramına dayanmasıdır. Bunu daha iyi anlamak için, Shapley değerlerinin genel bir özelliğine bakabiliriz; bu özellik, Shapley değerlerinin toplamının, bizim durumumuzda katkıların toplamının, $\nu(T)-\nu(\{\})$'ye eşit olduğunu belirtir. Burada, $\nu$ bir küme fonksiyonudur (bizim durumumuzda, attribution_func'ın beklentisi) ve $T$ tüm oyuncuların tam kümesidir (bizim durumumuzda, tüm gürültü değişkenleri).

Şimdi, ortalamayı kullanırsak, $\nu(T)$ $\mathbb{E}_{\mathbf{N}}[\mathbb{E}[Y \mid \mathbb{N}]]=\mathbb{E}[Y]$ olur, çünkü hedef değişken $Y$ grafiksel nedensel modeldeki tüm gürültü değişkenleri $\mathbf{N}$'ye kesin olarak bağlıdır. Benzer şekilde, $\nu(\{\})$ $\mathbb{E}[Y \mid\{\}]=\mathbb{E}[Y]$ olur. Bu, $\mathbb{E}_{\mathbb{N}}[\mathbb{E}[Y \mid \mathbb{N}]]-\mathbb{E}[Y \mid\{\}]=0$ ile sonuçlanır, yani ortaya çıkan katkılar 0'a yakındır.

# 3.Kök Neden Analizi ve Açıklaması

DoWhy'ın grafiksel nedensel model çerçevesi, nedensel bir grafikteki düğümlerin tek nedensel mekanizmalara ayrıştırılması yoluyla, kök neden analizi ve gözlenen etkilerin açıklanması için güçlü araçlar sunar.

**Öznitelik Anomalileri**: Anormal gözlemler (yani aykırı değerler) verildiğinde, bunları bunlara neden olan düğümlere odaklanabiliriz. Buradaki fikir, karşıolgusal (counterfactual) soruyu sormaktır: "Eğer düğüm x farklı davransaydı, yine de anomaliyi gözlemler miydik?". Hedef düğümümüzdeki değişiklik, bir düğümün gözlemlenen anomaliye ne kadar katkıda bulunduğunu niceliksel olarak belirler. Adil bir öznitelik sağlamak için, bunun düğümlerdeki değişikliklerin farklı kombinasyonları için sistematik olarak tahmin edilmesi gerekir.

**Dağılım Değişikliği Değişkeni**: Dağılımın değiştiği iki veri kümesi verildiğinde, değişiklikleri, değişikliklere neden olan grafikteki düğümlere atfediyoruz. Burada, önce değişen düğümlerin veri üretme mekanizmalarını belirliyoruz ve sonra dağılımdaki değişiklikleri (örneğin, varyanstaki veya KL sapmasındaki bir değişiklik açısından) bu mekanizmalara atfediyoruz.

**Öznitelik İlgililiği**: Burada, bir özniteliğin bir model için ne kadar ilgili olduğu sorusunu ele alıyoruz. SHAP gibi popüler paketler, belirli bir küme fonksiyonu tanımlayarak ve her girdi özniteliği için Shapley değerini tahmin ederek bu soruyu ele alır. Bizim durumumuzda, benzer bir şey yapıyoruz, ancak nedensel bir mekanizmanın olasılıksal doğasını da dahil ediyoruz. Yani, bir değişkeni etkileyen gürültüyü de dahil ediyoruz. Bununla, girdilerin ilgililiğini (relevance) hesaplayabiliriz, ancak gürültü tarafından temsil edilen açıklanamayan kısmı da dahil edebiliriz. Ayrıca, DoWhy'daki Shapley değer tahmincisi, özelleştirilmiş bir küme fonksiyonu tanımlamak için esnek bir yol sunar.

## 3.1. Öznitelik Anomalileri

İlgi duyduğumuz bir hedef düğümde bir anormallik gözlemlediğimizde şu soruyu ele alabiliriz:

Yukarı akıştaki düğümlerin her biri ve hedef düğüm, gözlemlenen anormalliğe ne kadar katkıda bulundu?

Bunun aracılığıyla, her düğümün anormal gözleme katkısını belirliyor ve belirli bir şekilde ölçüyoruz. Bu yöntem şu makaleye dayanmaktadır:

Kailash Budhathoki, Lenon Minorics, Patrick Blöbaum, Dominik Janzing. Causal structure-based root cause analysis of outliers International Conference on Machine Learning, 2022

### 3.1.1. Kullanımı

Öncelikle basit bir X → Y → Z → W zincirinin bazı örnek verilerini üretelim:

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import networkx as nx
from dowhy import gcm

In [2]:
X = np.random.uniform(low=-5, high=5, size=1000)
Y = 0.5 * X + np.random.normal(loc=0, scale=1, size=1000)
Z = 2 * Y + np.random.normal(loc=0, scale=1, size=1000)
W = 3 * Z + np.random.normal(loc=0, scale=1, size=1000)
data = pd.DataFrame(data=dict(X=X, Y=Y, Z=Z, W=W))

Sonra, neden-sonuç ilişkilerini *tersine çevrilebilir* yapısal nedensel bir model olarak modelliyoruz ve bunu verilere uyarlıyoruz. Nedensel mekanizmaları otomatik olarak atamak için `auto` modülü kullanıyoruz:

In [3]:
causal_model = gcm.InvertibleStructuralCausalModel(nx.DiGraph([('X', 'Y'), ('Y', 'Z'), ('Z', 'W')]))
gcm.auto.assign_causal_mechanisms(causal_model, data)
gcm.fit(causal_model, data)

Fitting causal mechanism of node W: 100%|██████████| 4/4 [00:00<00:00, 326.77it/s]


Sonra, bir anormallik yaratırız. Örneğin, $Y$'nin gürültüsünü alışılmadık derecede yüksek bir değere ayarlarız:

In [4]:
X = np.random.uniform(low=-5, high=5)  # Sample from its normal distribution.
Y = 0.5 * X + 5  # Here, we set the noise of Y to 5, which is unusually high.
Z = 2 * Y
W = 3 * Z
anomalous_data = pd.DataFrame(data=dict(X=[X], Y=[Y], Z=[Z], W=[W]))  # This data frame consist of only one sample here.

Burada, $Y$ kök nedendir ve bu da $Y, Z$ ve $W$'nin anormal olmasına yol açar. Artık ilgi duyduğumuz hedef düğümümüzün (örneğin, $W$) anomali skorlarını alabiliriz.

In [5]:
attribution_scores = gcm.attribute_anomalies(causal_model, 'W', anomaly_samples=anomalous_data)
attribution_scores

Evaluate set function: 16it [00:00, ?it/s]


{'X': array([0.3567913]),
 'Y': array([0.95439735]),
 'Z': array([0.14621173]),
 'W': array([0.1374822])}

Burada doğrusal bir ilişki kullansak da, yöntem keyfi doğrusal olmayan ilişkileri de barındırabilir.
Ayrıca birden fazla temel neden de olabilir.

Sonuçların yorumlanması: $W$'nin kendisi de dahil olmak üzere $W$'nin atalarının gözlemlenen anomaliye katkısını tahmin ettik. Tüm düğümler bir dereceye kadar katkıda bulunsa da, $Y$ öne çıkanıdır. $Z$'nin de $Y$ nedeniyle anormal olduğunu, ancak yalnızca yüksek değeri $Y$'den miras aldığını unutmayın. Attribution yöntemi bunu belirleyebilir ve $Y$ ve $Z$'nin katkılarını ayırt edebilir. Negatif bir katkı durumunda, karşılık gelen düğüm bir anomalinin olasılığını bile azaltır, yani görünürdeki ciddiyetini azaltır.

Anomali skorunun ayrıntılı bir yorumu için, başvurulan makaleye bakın. Aşağıdaki bölüm ayrıca bazı sezgiler sunar.

### 3.1.2. Yöntemin Anlaşılması

Bu yöntemde, belirli bir gözleme yol açan gürültüyü yeniden yapılandırmak ve değiştirmek için tersine çevrilebilir nedensel mekanizmalar kullanırız. Daha sonra, "Belirli bir düğümün gürültü değeri 'normal' dağılımından olsaydı, hedef düğümde yine de anomali bir değer gözlemler miydik?" diye sorarız. Öğrenilmiş dağılımına göre, yukarı akış gürültü değişkeninin değerini değiştirdikten sonra hedef düğümdeki anomalinin şiddetindeki değişiklik, düğümün anomaliye katkısını gösterir. Gürültü değerini gerçek düğüm değerine göre kullanmanın avantajı, yalnızca düğümden kaynaklanan etkiyi ölçmemiz ve ebeveynlerinden miras almamamızdır.

Süreç aşağıdaki şekilde özetlenebilir:

<div>
<img src="../img/attribute_anomalies.png" width="250"/>
</div>

1. Hedef değişken $X_n$ için bir anomali skorunu $S\left(x_n\right):=-\log P\left(g\left(X_n\right) \geq g\left(x_n\right)\right)$ olarak tanımlarız. Burada, $\boldsymbol{g}$ bir isolation forest, medyan/ortalama fark veya bir anomali skoru veren herhangi bir başka model gibi keyfi bir anomali skorlayıcısıdır. Bir $x_n$ gözlemi için bilgi teorik skoru $S\left(x_n\right)$ daha sonra ölçek değişmezdir (scale invariant) ve anomali skorlayıcısının seçiminden bağımsızdır.
2. Düğüm $j$'nin katkısını $\log \frac{P\left(g\left(X_n\right) \geq g\left(x_n\right) \mid \text { tüm gürültü değerlerini değiştir } n_1, \ldots, n_{j-1} \text { rassal değerlerle }\right)}{P\left(g\left(X_n\right) \geq g\left(x_n\right) \mid \text { tüm gürültü değerlerini değiştir } n_1, \ldots, n_j \text { rassal değerlerle }\right)}$ şeklinde tanımlarız, burada 'rassal değerler' öğrenilen gürültü dağılımına göredir. Bu log-oran, gürültü $N_j$'yi rassal hale getirmenin anomali olayının olasılığını nasıl azalttığını ölçer.
3. Düğüm yeniden sıralamasından kaynaklanan belirsizliği ortadan kaldırmak için katkıyı tüm sıralamalar arasında simetrik hale getiririz. Bunun için bir Shapley simetrizasyonu kullanırız.
4. Son Shapley değerleri, anomali skorları $S\left(x_n\right)$'a toplanır.

Bu yaklaşımın önemli özelliği, yalnızca nadir olayların yüksek katkılar elde edebilmesidir; yaygın olaylar nadir olanları açıklayamaz.

Basit Örnek: Katkıları doğrudan elde edebileceğimiz basit ve anlaşılır bir örnek, dört kenarı olan bir zar ve 100 kenarı olan bir zar (örneğin, bir D\&D oyunundan) olduğunda ortaya çıkar. Zarı atarsak ve $(1,1)$ gelirse, bu olay için 'aykırı değer' puanı $\log (4 \cdot 100)=\log (4)+\log (100)$ olur. Burada, her zarın bu olaya katkısı doğrudan toplama dayanır; dört kenarlı zar $23 \%$ katkıda bulunurken, 100 kenarlı zar $(1,1)$ olayına $77 \%$ katkıda bulunmuştur.

## 3.2.Dağılımsal Değişikliklerin İlişkilendirilmesi

Dağılım değişikliklerini ilişkilendirirken şu soruyu yanıtlarız:

İki veri seti arasında sistemimdeki hangi mekanizma değişti? Ya da başka bir deyişle, verilerimdeki hangi düğüm farklı davranıyor?

Burada, nedensel mekanizmanın değiştiği grafikteki düğümü veya düğümleri tanımlamak istiyoruz. Örneğin, bir mikro servis mimarisi içindeki uygulamamızın gecikme süresinde bir artış tespit edersek, davranışı değişen düğümü / bileşeni tanımlamayı amaçlıyoruz. DoWhy, bir dağılımdaki değişiklikleri, aşağıdaki makaleyi takip eden yukarı akış (upstream) düğümlerinin nedensel mekanizmalarındaki değişikliklere tanımlamak ve atfetmek için bir yöntem uygular:

Kailash Budhathoki, Dominik Janzing, Patrick Blöbaum, Hoiyi Ng. Why did the distribution change? Proceedings of The 24th International Conference on Artificial Intelligence and Statistics, PMLR 130:1666-1674, 2021.

### 3.2.1. Kullanımı

Yöntemin nasıl kullanılacağını görmek için yukarıdaki mikro servis örneğini ele alalım ve her biri gecikmeleri izleyen dört servisten oluşan bir sistemimiz olduğunu varsayalım $X, Y, Z, W$. Yeni bir dağılım yapmayı ve dağılımdan önceki ve sonraki gecikmeleri kaydetmeyi planladığımızı varsayalım. Dağılımdan önce toplanan gecikme verilerine data_old ve dağılımdan sonra toplanan verilere data_new diyeceğiz:

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import networkx as nx
from dowhy import gcm
from scipy.stats import halfnorm

In [2]:
X = halfnorm.rvs(size=1000, loc=0.5, scale=0.2)
Y = halfnorm.rvs(size=1000, loc=1.0, scale=0.2)
Z = np.maximum(X, Y) + np.random.normal(loc=0, scale=0.5, size=1000)
W = Z + halfnorm.rvs(size=1000, loc=0.1, scale=0.2)
data_old = pd.DataFrame(data=dict(X=X, Y=Y, Z=Z, W=W))

In [3]:
X = halfnorm.rvs(size=1000, loc=0.5, scale=0.2)
Y = halfnorm.rvs(size=1000, loc=1.0, scale=0.2)
Z = X + Y + np.random.normal(loc=0, scale=0.5, size=1000)
W = Z + halfnorm.rvs(size=1000, loc=0.1, scale=0.2)
data_new = pd.DataFrame(data=dict(X=X, Y=Y, Z=Z, W=W))

Burada, çok iş parçacıklı kodun ardışık bir koda yanlışlıkla dönüştürülmesini simüle eden $Z$'nin davranışını değiştiriyoruz (paralel olarak $X$ ve $Y$'yi beklemek yerine bunları ardışık olarak beklemek). Bu, $Z$'nin ve ardından $W$'nin dağılımını değiştirecektir.

Sonra, neden-sonuç ilişkilerini olasılıksal bir nedensel model olarak modelleyeceğiz:

In [4]:
causal_model = gcm.ProbabilisticCausalModel(nx.DiGraph([('X', 'Z'), ('Y', 'Z'), ('Z', 'W')])) # (X, Y) -> Z -> W
gcm.auto.assign_causal_mechanisms(causal_model, data_old)

Son olarak, $W$ dağılımlarındaki değişiklikleri nedensel mekanizmalardaki değişikliklere bağlıyoruz:

In [6]:
attributions = gcm.distribution_change(causal_model, data_old, data_new, 'W')
attributions

Evaluate set function: 16it [00:00, ?it/s]


{'W': 0.046919664317514445,
 'X': 0.010759151751588633,
 'Y': -0.022628749049083765,
 'Z': 0.7739102109556353}

$W$ dağılımı da değişmiş olsa da, yöntem değişikliği neredeyse tamamen $Z$'ye ilişkilendirirken diğer değişkenler için önemsiz skorlar verir. Bu, yalnızca $\boldsymbol{Z}$ mekanizmasını değiştirdiğimiz için beklentilerimizle uyumludur. Skorların biriminin kullanılan ölçüye bağlı olduğunu unutmayın (bir sonraki bölüme bakın).

Okuyucunun fark etmiş olabileceği gibi, bu yöntemi kullanırken herhangi bir uydurma adımı (fitting step) söz konusu değildir. Bunun nedeni, bu fonksiyonun dahili olarak uydurma fonksiyonunu çağıracak olmasıdır. Daha kesin olmak gerekirse, bu fonksiyon nedensel grafiğin iki kopyasını yapacak ve bir grafiği ilk veri kümesine, ikinci grafiği ise ikinci veri kümesine uyduracaktır.

### 3.2.2. Yöntemin Anlaşılması

Bu yöntemin arkasındaki fikir, eski veri kümesine dayanarak öğrenilen nedensel mekanizmayı **sistematik** olarak yeni veri kümesine dayanarak öğrenilen mekanizmayla değiştirmektir. Her değiştirmeden sonra, veri oluşturma sürecinin eski ve yeni mekanizmaların bir karışımı olduğu hedef düğüm için yeni örnekler üretilir. Amacımız, hedefin farklı bir marjinal dağılımına yol açacak şekilde değişen mekanizmaları belirlemektir, değişmeyen mekanizmalar ise aynı marjinal dağılıma yol açacaktır. Bunu başarmak için, mekanizmaları sistematik olarak değiştirmek için Shapley simetrileştirmesi fikrini kullanırız. Bu, hangi düğümlerin değiştiğini belirlememizi ve bazı ölçütlere göre bir ilişki skoru tahmin etmemizi sağlar. Mekanizmadaki bir değişikliğin, temel modeldeki fonksiyonel bir değişiklikten veya (gözlemlenmeyen) gürültü dağılımındaki bir değişiklikten kaynaklanabileceğini unutmayın. Ancak, her iki değişiklik de mekanizmada bir değişikliğe yol açacaktır.

Adımlar aşağıdaki şekilde özetlenebilir:

<div>
<img src="../img/dist_change.png" width="500"/>
</div>

1. 'Eski' verilerden (örneğin, dağılımdan önceki gecikmeler (latencies)) koşullu dağılımları tahmin ederiz: $P_{X_1, \ldots, X_n}=\prod_j P_{X_j \mid P A_{j^{\prime}}}$ burada $P_{X_j \mid P A_j}$, $X_j$ düğümünün nedensel mekanizmasıdır ve $P A_j$, $X_j$ düğümünün ebeveynleridir
2. 'Yeni' verilerden (örneğin, dağılımdan sonraki gecikmeler) koşullu dağılımları tahmin ederiz:

$$
\tilde{P}_{X_1, \ldots, X_n}=\prod_j \tilde{P}_{X_j \mid P A_j}
$$
3. 'Eski' verilere dayalı mekanizmaları sistematik olarak, teker teker 'yeni' verilere dayalı mekanizmalarla değiştiririz. Bunun için, $P_{X_j \mid P A_j}$'yi her $j$ için $\tilde{P}_{X_j \mid P A_j}$ ile değiştiririz. $T \subseteq\{1, \ldots, n\}$'deki düğümler daha önce değiştirilmişse, $\tilde{P}_T^{X_n}=\sum_{x_1, \ldots, x_{n-1}} \prod_{j \in T} \tilde{P}_{X_j \mid P A_j} \prod_{j \notin T} P_{X_j \mid P A_j}$ düğümü $n$ için yeni bir marjinal elde ederiz.
4. Shapley değerlerini kullanarak $T$ marjinalindeki değişimi $P_{T \bigcup\{j\}}^{X_n}$ ve $P_T^{X_n}$'yi karşılaştırarak $X_j$'ye ilişkilendiririz. Burada, değişimi yakalamak için orijinal dağılıma göre KL sapması veya varyanslardaki fark gibi farklı ölçüler kullanabiliriz.

Daha ayrıntılı açıklama için ilgili makaleye bakın: https://proceedings.mlr.press/v130/budhathoki21a/budhathoki21a.pdf

## 3.3.Öznitelik İlgililiği

Öznitelik ilgililiği bağlamında, şu soruyu ele alıyoruz:

Bir öznitelik, hedef değişken için ne kadar alakalıdır?

Bu soruyu ele almak için birden fazla yaklaşım vardır. Bu bağlamda, eldeki soruna bağlı olarak değiştirilebilen, özel olarak uyarlanmış küme fonksiyonlarına sahip Shapley değerleri kavramını kullanıyoruz. En yaygın ayarda, bu SHAP paketine benzer. Ancak, küme fonksiyonunu değiştirmek için basit ve esnek bir yöntem sunuyoruz, böylece açıklamaları özelleştiriyoruz.

Gürültünün (yani açıklanamayan) bir modelin girdi özniteliklerinden biri olarak da dahil edildiği nedensel bir çerçeveyle buna yaklaşıyoruz. Başka bir deyişle, bir düğümün ebeveynlerini ve gürültüyü girdi olarak içeren nedensel mekanizmayı inceleyebiliriz.

Shapley değerlerinin belirli bir modelin nedensel açıklamaları olarak nasıl yorumlanabileceğine ilişkin daha fazla bilgi için aşağıdaki makaleye bakın:

Dominik Janzing, Lenon Minorics, Patrick Blöbaum. Feature relevance quantification in explainable AI: A causal problem International Conference on Artificial Intelligence and Statistics, 2907-2916, 2021.

### 3.3.1. Kullanımı

Nedensel olmayan kullanım durumları da olsa ve araçlarımız bunlar için de kullanılabilse de, öncelikle bir düğümün nedensel mekanizmasını kullandığımız bir örneğe bakacağız. Her zamanki gibi, önce bazı örnek veriler üreteceğiz:

In [7]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import networkx as nx
from dowhy import gcm

In [8]:
X = np.random.normal(loc=0, scale=1, size=1000)
Z = np.random.normal(loc=0, scale=1, size=1000)
Y = X + 3 * Z + np.random.normal(loc=0, scale=1, size=1000)
data = pd.DataFrame(data=dict(X=X, Y=Y, Z=Z))

Şimdi, bunu analiz etmek için farklı seçenekler var. Bunu başlangıçta nedensel bir model olarak modelleyelim:

In [9]:
causal_model = gcm.InvertibleStructuralCausalModel(nx.DiGraph([('X', 'Y'), ('Z', 'Y')])) # X -> Y <- Z
gcm.auto.assign_causal_mechanisms(causal_model, data)
gcm.fit(causal_model, data)

Fitting causal mechanism of node Z: 100%|██████████| 3/3 [00:00<00:00, 300.08it/s]


Artık ebeveynlerin global önemini (popülasyon/dağılım düzeyi) değerlendirebiliriz:

In [10]:
parent_relevance, noise_relevance  = gcm.parent_relevance(causal_model, target_node='Y')
parent_relevance, noise_relevance

Evaluate set function: 8it [00:00, ?it/s]


({('X', 'Y'): 0.8936534491130332, ('Z', 'Y'): 8.878179249310966},
 array([0.98738613]))

Global ilgi bağlamında, girdilerin ilgi düzeyi varsayılan olarak varyansa katkılarına göre değerlendirilir. Burada, ebeveyn $\boldsymbol{Z}$'nin açıkça en ilgili öznitelik olduğunu görebiliriz. Ayrıca, bu durumda $X$'in ilgi düzeyine eşdeğer olan gürültünün ilgi düzeyini de görebiliriz.

Yukarıdaki örnekte, nedensel yapıyı doğrudan kullandık ve global ilgi düzeyini tahmin ettik. Ancak, bu yöntemi bir kara kutu tahmincisi ile de kullanabiliriz. Bunun için, bir örnek olarak doğrusal bir model kullanalım:

In [11]:
from sklearn.linear_model import LinearRegression
from dowhy.gcm.util.general import variance_of_deviations

In [13]:
mdl = LinearRegression()
mdl.fit(data[['X', 'Z']].to_numpy(), Y)
relevance = gcm.feature_relevance_distribution(mdl.predict, data[['X', 'Z']].to_numpy(), subset_scoring_func=variance_of_deviations)
relevance

Evaluate set function: 4it [00:00, ?it/s]


array([0.83917095, 8.79396512])

Bu durumda, alt küme skorlama fonksiyonumuzu doğrudan sapmaların varyansı olarak tanımladık ve benzer sonuçlar elde ettik.

Son olarak, tüm popülasyon yerine bireysel bir gözlem için ilgi düzeyine bakalım. Bunun için, aşağıdaki gözlem için ilgi düzeyini açıklamak istediğimizi varsayalım:

In [14]:
single_observation = np.array([[2, 1]])

Burada, $X=2$ ve $Z=1$, sonuç olarak $Y=5$. Bu tek gözlemin ilgi düzeyini değerlendirelim:

In [15]:
from dowhy.gcm.util.general import means_difference

In [16]:
relevance = gcm.feature_relevance_sample(mdl.predict, data[['X', 'Z']].to_numpy(), baseline_samples=single_observation, subset_scoring_func=means_difference)
relevance

Evaluate set function: 4it [00:00, ?it/s]


array([[1.89380021, 3.01183844]])

Beklendiği gibi, $X$, $Z_t$'den daha büyük bir değere sahip olsa da, katsayısı nedeniyle $Z$ modelin sonucu için daha önemlidir.

Not: Diğer yöntemlere benzer şekilde, yöntemi burada doğrusal bir ilişki kullanarak gösterip doğrularken, keyfi doğrusal olmayan ilişkileri de işleyebilir.

# 4."...Olursa Ne Olurdu?" Sorularını Sormak ve Cevaplamak

Grafiksel nedensel modeller kullanılarak, Pearl'ün nedensellik merdiveninde tanımlandığı gibi hem basamak 2 (müdahaleler-interventions) hem de basamak 3 (karşıt olgular-counterfactuals) sorularını ele almak mümkündür.

## 4.1.Müdahalelerin Etkisini Simüle Etmek

Müdahalelerin etkisini simüle ederek şu tür sorulara cevap veriyoruz:

Y'ye müdahale edersem Z değişkenine ne olur?

### 4.1.1. Kullanımı

Yöntemin nasıl çalıştığını görmek için biraz veri üretelim:

In [17]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import networkx as nx
from dowhy import gcm

In [18]:
X = np.random.normal(loc=0, scale=1, size=1000)
Y = 2*X + np.random.normal(loc=0, scale=1, size=1000)
Z = 3*Y + np.random.normal(loc=0, scale=1, size=1000)
training_data = pd.DataFrame(data=dict(X=X, Y=Y, Z=Z))

Daha sonra, neden-sonuç ilişkilerini olasılıksal bir nedensel model olarak modelleyeceğiz ve bunu verilere uyduracağız:

In [19]:
causal_model = gcm.ProbabilisticCausalModel(nx.DiGraph([('X', 'Y'), ('Y', 'Z')])) # X -> Y -> Z
gcm.auto.assign_causal_mechanisms(causal_model, training_data)
gcm.fit(causal_model, training_data)

Fitting causal mechanism of node Z: 100%|██████████| 3/3 [00:00<00:00, 749.12it/s]


Son olarak, $X$ üzerinde bir müdahale gerçekleştirelim. Burada, $d o(X:=1)$ müdahalesini açıkça gerçekleştiriyoruz:

In [20]:
samples = gcm.interventional_samples(causal_model,
                                     {'X': lambda x: 1},
                                     num_samples_to_draw=1000)
samples.head()

,X,Y,Z
0,1,3.033108,10.298888
1,1,2.811108,9.730742
2,1,0.493623,1.423954
3,1,2.382729,7.329285
4,1,0.581403,1.556154


Gördüğümüz gibi, X artık sabit bir 1 değerinde sabitlendi. Bu, **atomik müdahale** olarak bilinir. Ayrıca, rassal değişken X'i herhangi bir değerle kaydırdığımız kaydırma müdahaleleri de gerçekleştirebiliriz:

In [21]:
samples = gcm.interventional_samples(causal_model,
                                     {'X': lambda x: x + 0.5},
                                     num_samples_to_draw=1000)
samples.head()

,X,Y,Z
0,1.372186,3.930829,12.960721
1,0.265933,-0.126947,0.687153
2,0.396088,0.324487,0.440289
3,-1.147428,-1.940989,-6.799381
4,1.351232,2.199626,6.996288


## 4.2.Karşı-Olguların Hesaplanması

Karşı-olguları hesaplayarak şu soruyu yanıtlıyoruz:

Değişken $\boldsymbol{X}$'in $\boldsymbol{x}$ değerine ayarlandığı bir değişken $\boldsymbol{Z}$ için belirli bir sonuç $\boldsymbol{z}$ gözlemledim. $X$'e farklı bir $x^{\prime}$ değeri atamak için müdahale etseydim, $Z$ değerine ne olurdu?

Somut bir örnek olarak, aşağıdakileri hayal edebiliriz:

Kolesterol LDL'mde sağlıksız yüksek seviyeler görüyorum $(Z=10)$. Son aylarda buna karşı hiçbir ilaç almadım $(x=0)$. Günde 5 g'lık bir ilaç dozu $(X:=5)$ almış olsaydım, kolesterol LDL seviyeme ( $\boldsymbol{Z}$) ne olurdu?

Pearl’ün grafiksel nedensel model çerçevesine dayalı karşı olguların tahmininin, müdahalesel örneklerin oluşturulmasından daha güçlü varsayımlar gerektirdiğini unutmayın.

### 4.2.1. Kullanımı

Bu yöntemi nasıl kullanabileceğimizi görmek için biraz veri üretelim:

In [22]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import networkx as nx
from dowhy import gcm

In [23]:
X = np.random.uniform(low=0, high=10, size=2000)
Y = -2*X + np.random.normal(loc=0, scale=5, size=2000)
Z = 3*Y + 80 + np.random.normal(loc=0, scale=5, size=2000)
training_data = pd.DataFrame(data=dict(X=X, Y=Y, Z=Z))

Kolesterol örneğini düşünürsek, burada $Y$'yi kolesterol LDL seviyesini etkileyen bir tür vücut ölçümü olarak düşünebiliriz, örneğin vücudun ilacı emme yeteneğinin veya metabolizmasının bir türevi.

Sonra, neden-sonuç ilişkilerini modelleyeceğiz ve modelleri verilere uyduracağız. Karşı-olguları tahmin etmek, toplamsal gürültü modelleri ile temsil edilebilen tersinir (invertible) bir SCM gerektirir:

In [24]:
causal_model = gcm.InvertibleStructuralCausalModel(nx.DiGraph([('X', 'Y'), ('Y', 'Z')])) # X -> Y -> Z
gcm.auto.assign_causal_mechanisms(causal_model, training_data)
gcm.fit(causal_model, training_data)

Fitting causal mechanism of node Z: 100%|██████████| 3/3 [00:00<00:00, 719.93it/s]


$x=0, y=5, z=110$ değerlerini gözlemlediğimizi varsayalım. İlaç almanın, örneğin $x:=5$ olarak ayarlamanın $z$ için daha küçük bir değere yol açıp açmayacağını bilmekle ilgileniyoruz. Bu senaryo için $z$'nin karşı-olgusal değeri aşağıdaki şekilde tahmin edilebilir:

In [25]:
gcm.counterfactual_samples(
    causal_model,
    {'X': lambda x: 5},
    observed_data=pd.DataFrame(data=dict(X=[0], Y=[5], Z=[110])))

,X,Y,Z
0,5,-4.774785,80.591146


Gördüğümüz gibi, $X$ tedavi/müdahale değerimizi 5 olarak alır ve $Y$ ve $Z$ eğitilmiş nedensel modellerimize ve sabit gözleme dayalı kesin değerler alır. Örneğin, $X$ 0 ve $Y$ 5 olsaydı, veri oluşturma sürecine dayanarak $Z$'nin 95 olmasını beklerdik. Ancak, $Z$'nin 110 olduğunu gözlemledik, bu da bu özel örnekte yaklaşık $\sim 15$'lik bir gürültü değeri olduğunu gösteriyor. Bu gizli gürültü faktörünü bildiğimizde, $X$'i 5 olarak ayarlasaydık $Z$'nin karşıt olgusal değerini tahmin edebiliriz, bu da yukarıdaki sonuçta gösterildiği gibi yaklaşık $\sim 80$'dir.

Ayrıca bu gürültü değerlerini doğrudan fonksiyona sağlayabiliriz:

In [26]:
gcm.counterfactual_samples(
    causal_model,
    {'X': lambda x: 5},
    noise_data=pd.DataFrame(data=dict(X=[0], Y=[5], Z=[15])))

,X,Y,Z
0,5,-5.113236,79.653499


Gördüğümüz gibi, $X$ 5 ve $y=-2 \cdot x+5=-5 olarak ayarlandığında, z$ yaklaşık $\sim 65$ olmalıdır. Ancak $Z$ için gizli gürültünün yaklaşık $\sim 15$ olduğunu biliyoruz. Dolayısıyla karşıt olgusal sonuç yine $z=3 * y+80+15=80$ olur.

Elde ettiğimiz tahmini değerlerin sonuçlarda gördüğümüz gibi kesin olmadığını belirtmek önemlidir, çünkü model parametreleri mükemmel bir şekilde öğrenilmemiştir. Özellikle, $Y$ ve $Z$ için öğrenilen katsayılar, veri oluşturma sürecinin önerdiği gibi sırasıyla tam olarak -2 ve 3 değildir. Yapay öğrenmede her zamanki gibi, daha fazla veri veya ince ayarlı modeller doğruluğu artırmaya yardımcı olabilir.

### 4.2.2. Yöntemin Anlaşılması

Grafiksel nedensel modellerdeki karşı olgular, Müdahalelerin Etkisini Simüle Etmeye çok benzer, ancak önemli bir fark vardır: ***müdahaleleri gerçekleştirirken geleceğe bakarız, karşı olgular içinse alternatif bir geçmişe bakarız.*** Bunu hesaplamada yansıtmak için, müdahaleleri gerçekleştirirken, önce belirli bir gözlem için tüm gürültü değerlerini yeniden yaratırız ve sonra karşı olgu sonucunu tahmin ederiz. Ancak bu, müdahale örnekleri oluşturmaktan daha güçlü modelleme varsayımları gerektirir.

Bir düğüm $Y$'nin nedensel mekanizmasının $Y:=f(X, N)$ şeklinde gösterildiğini hatırlayın; burada $X$, $Y$'nin ebeveynleridir ve $N$ gürültüdür. $Y$ için müdahaleci örnekler üretmek için $N$'den herhangi bir rassal değer alabiliriz; ancak karşı olgular için, önce gözlemimize yol açan belirli gürültü değerini (modele dayalı olarak) yeniden oluşturmamız gerekir. Bu, nedensel mekanizmanın $N$ açısından tersine çevrilebilir olmasını gerektirir; ancak $X$ açısından tersine çevrilebilir olması gerekmez. Bunu sağlayan yaygın bir modelleme varsayımı, gürültünün $N=Y-f(X)$ ile yeniden oluşturulabildiği $Y:=f(X)+N$ biçimindeki eklemeli gürültü modelleridir. Şu anda karşıt olgusal tahmin için yalnızca sürekli verilerin desteklendiğini, müdahaleci örnekler üretmek için veri türünde herhangi bir kısıtlama olmadığını unutmayın.

Gürültünün buradaki rolünü daha da açıklığa kavuşturmak için, giriş bölümündeki yüksek kolesterol LDL seviyeleriyle ilgili örneğe geri dönelim. Egzersiz ve genetik gibi kolesterol seviyelerini etkileyebilecek birçok gözlemlenmemiş faktör olduğunu gördüğümüzde, şu soru ortaya çıkıyor: İlaç alsaydım, LDL seviyelerim düşer miydi? Bunu cevaplamak için, konuya özgü gözlemlenmemiş faktörleri (yani gürültüyü) sabit tuttuğumuz ve yalnızca varsayımsal olarak alınan ilaç miktarını değiştirdiğimiz müdahaleleri kullanabiliriz. Pratikte bu, önce gürültüyü yeniden yapılandırarak ve ardından ilaç miktarına müdahale ettikten sonra LDL seviyelerini tahmin etmek için bu belirli değeri kullanarak elde edilebilir. Burada, gürültü dağılımından rassal örnekleme yapmak yerine yeniden yapılandırılmış gürültü değerini kullanmak çok önemlidir. Aksi takdirde, ilacın kendisinden değil, tesadüfen düşük seviyelere neden olan üretilen gürültü değerinden dolayı LDL seviyelerinde bir azalma görülebilir. Modelleme varsayımlarının yaklaşık olarak doğru olduğunu varsayarak, ilacın karşıt olgusal senaryoda yardımcı olup olmayacağını analiz edebiliriz.

Tersine çevrilebilir mekanizmalar hakkında açıklama: Genellikle, $N$ açısından tersine çevrilebilir mekanizmalar nokta karşıt olguları tahmin etmemize izin verir. Ancak, bazı mekanizmaların tersine çevrilemez olmasına da izin vermek mümkündür. Ancak bu durumda, gözlemsel kanıtlara dayalı bir karşıt olgu dağılımı elde ederiz; bu, mutlaka nokta bazında olmayabilir.

# 5.Dağılım Dışı Girdiler için Çıktının Tahmin Edilmesi

DoWhy tahmin algoritmalarını da destekler. Tüm özellikleri kullanan standart yapay öğrenme algoritmalarının aksine, DoWhy'daki tahmin algoritması tahmin için nedensel bir özellik gösterimi öğrenmeyi amaçlar. Gösterim, kullanıcı tarafından sağlanan yardımcı özelliklerin alan bilgisi kullanılarak öğrenilir.